# ALL installs 

In [1]:
# ! pip install requests
# ! pip install kafka-python
# ! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark ... done


In [1]:
!pip freeze > requirements.txt


# ALL imports

In [2]:
from allsecrets import *
import json,requests
from json import dumps,loads 
#dumps converts python object to json string
#loads converts json string to python object
from kafka import KafkaProducer, KafkaConsumer



In [3]:
from pyspark.sql import SparkSession , functions as F

from pyspark.sql.functions import udf,col,countDistinct,date_format,row_number

from pyspark.sql.types import FloatType

from pyspark.sql.window import Window 

spark = SparkSession.builder.appName('OMDB')\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


22/11/14 16:52:51 WARN Utils: Your hostname, AMRIT resolves to a loopback address: 127.0.1.1; using 172.17.124.125 instead (on interface eth0)
22/11/14 16:52:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/14 16:52:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Json print function

In [ ]:
def json_print(json_object):
    text = json.dumps(json_object, sort_keys=True, indent=4) # sort_keys=True means sort the keys in alphabetical order
    # indent 4 means 4 spaces for each indentation
    print(text)

## OMDB dataset

In [ ]:
# http://www.omdbapi.com/?i=tt3896198&apikey=******* # api key for    OMBD API

parameters = {'t':'top gun maverick', "apikey": ombd_api} 
OMDB_response = requests.get("http://www.omdbapi.com/", params=parameters)
print(OMDB_response.status_code)
json_print(OMDB_response.json())

200
{
    "Actors": "Tom Cruise, Jennifer Connelly, Miles Teller",
    "Awards": "2 wins & 9 nominations",
    "BoxOffice": "$716,498,761",
    "Country": "United States",
    "DVD": "N/A",
    "Director": "Joseph Kosinski",
    "Genre": "Action, Drama",
    "Language": "English",
    "Metascore": "78",
    "Plot": "After thirty years, Maverick is still pushing the envelope as a top naval aviator, but must confront ghosts of his past when he leads TOP GUN's elite graduates on a mission that demands the ultimate sacrifice from those chosen to ...",
    "Poster": "https://m.media-amazon.com/images/M/MV5BZWYzOGEwNTgtNWU3NS00ZTQ0LWJkODUtMmVhMjIwMjA1ZmQwXkEyXkFqcGdeQXVyMjkwOTAyMDU@._V1_SX300.jpg",
    "Production": "N/A",
    "Rated": "PG-13",
    "Ratings": [
        {
            "Source": "Internet Movie Database",
            "Value": "8.4/10"
        },
        {
            "Source": "Rotten Tomatoes",
            "Value": "96%"
        },
        {
            "Source": "Metacritic",

## Openweather 5 days 3 hour forcast

In [ ]:
#api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

parameters= {"lat": 27.708317, "lon": 85.3205817, "appid": open_weather_api}

Weather_response = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=parameters)
print(Weather_response.status_code)
json_print(Weather_response.json())

200
{
    "city": {
        "coord": {
            "lat": 27.7083,
            "lon": 85.3206
        },
        "country": "NP",
        "id": 7800827,
        "name": "Kathmandu",
        "population": 0,
        "sunrise": 1668386330,
        "sunset": 1668425252,
        "timezone": 20700
    },
    "cnt": 40,
    "cod": "200",
    "list": [
        {
            "clouds": {
                "all": 20
            },
            "dt": 1668427200,
            "dt_txt": "2022-11-14 12:00:00",
            "main": {
                "feels_like": 291.75,
                "grnd_level": 870,
                "humidity": 55,
                "pressure": 1018,
                "sea_level": 1018,
                "temp": 292.34,
                "temp_kf": 6.12,
                "temp_max": 292.34,
                "temp_min": 286.22
            },
            "pop": 0,
            "sys": {
                "pod": "n"
            },
            "visibility": 10000,
            "weather": [
            

# Test produces and consumer

## Test producer

In [ ]:
# zookeeper and kafka broker should be running in the background
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8'))
#value_serializer=lambda x: dumps(x).encode('utf-8') is used to convert the data into json format

# send the OMDB_response to the topic named OMDB
producer.send('OMDB', value=OMDB_response.json())

# send the Weather_response to the topic named Weather
producer.send('WEATHER', value=Weather_response.json())

# flush the messages to the topic
producer.flush() # flush() is a blocking function which ensures all the messages are sent to the topic


## Test consumer

In [8]:
consumer = KafkaConsumer('OMDB', bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest', enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))
#auto_offset_reset='earliest' means that the consumer will read from the beginning of the topic
#enable_auto_commit=True means that the consumer will commit the offset automatically so that it will not read the same message again
#value_deserializer=lambda x: loads(x.decode('utf-8')) is used to convert the data from json format


# consumer=KafkaConsumer('Weather', bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest', enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8')))


# for message in consumer:
#     message = message.value 
#     print(message)
#     print('')

In [11]:
# save consumed messages to a  spark dataframe
df = spark.createDataFrame(consumer)


KeyboardInterrupt: 

In [ ]:
df.show()

NameError: name 'df' is not defined